In [2]:
### For colab, run the following codes
from google.colab import drive
drive.mount('/content/gdrive')

## cd to your directory
%cd /content/gdrive/MyDrive/Work/PolygonObjectDetection-main

## cd to polygon-yolov5
%cd polygon-yolov5

## install requirements
!pip install -r requirements.txt

Mounted at /content/gdrive
/content/gdrive/MyDrive/Work/PolygonObjectDetection-main
/content/gdrive/MyDrive/Work/PolygonObjectDetection-main/polygon-yolov5
     |████████████████████████████████| 596 kB 5.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
# LOCAL ONLY
%cd /Users/antoniomorais/Work/PolygonObjectDetection/polygon-yolov5
!pip install -r requirements.txt

/Users/antoniomorais/Work/PolygonObjectDetection/polygon-yolov5


     |████████████████████████████████| 45.9 MB 26.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 31.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 26.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 17.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 14.8 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 28.6 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 35.7 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 38.9 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 17.3 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 37.3 MB/s eta 0:00:01


## Polygon Tutorial 2
Using data **segmentation** and module ***shapely*** to transform dataset to polygon boxes.
<br> COCO dataset as an example.

In [8]:
import json
import pycocotools
import shapely
import shapely.geometry
import glob
import numpy as np
import os

from pathlib import Path
from itertools import repeat
from tqdm import tqdm
from pycocotools import coco, mask

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

oneD2twoD = lambda x: [(x[2*i], x[2*i+1]) for i in range(len(x)//2)]   # one D [x, y, x, y, x, y, ...] to [(x, y), (x, y), ...]
catid_to_idx = {1: 0}


def seg2poly(dataset_path='',
             plot=True,):
    """
        Transform segmentation to polygon labels (x1, y1, x2, y2, x3, y3, x4, y4)
    """
    
    # Search for COCO json annotation files
    f = []  # json files
    p = Path(dataset_path)
    assert p.is_dir(), "'dataset_path' should be a valid path."
    f += glob.glob(str(p / '**' / '*.json'), recursive=True)
    assert f, f"Error: no searched annotations files (.json) with {dataset_path}"
    
    # Iterate through each json_file
    for json_file in f:
        coco_data = pycocotools.coco.COCO(json_file) # load coco data
        
        parent_path = Path(json_file).parent.parent
        img_txt = [] # store searched image files
        img_dir = parent_path / 'images'
        # Get prefix name
        for file in os.listdir(str(img_dir)):
            if file in json_file:
                prefix = file
                break
        
        img_dir = img_dir / prefix
        anno_dir = parent_path / 'labels' / prefix
        if not os.path.exists(str(anno_dir)): os.mkdir(str(anno_dir))
        
        print(f'Begin transformation for {prefix}')
        plot_now = 0
        for img_i, img in enumerate(tqdm(coco_data.dataset['images'])):
            
            plot = plot and plot_now<3 # test the first three images that are not crowded
            img_name = img_dir / img['file_name']
            if img_name.exists():
                anno_name = anno_dir / (os.path.splitext(img['file_name'])[0]+'.txt')
                anno_txt = [] # store label information

                if img['id'] not in coco_data.imgToAnns.keys(): continue
                img_txt.append(str(img_name)) # store current image file

                if plot: polygon_coords, segment_coords = [], [] # for plot

                # iterate through each object
                for object0 in coco_data.imgToAnns[img['id']]:

                    # if not crowded, use segmentation
                    if not object0['iscrowd']:
                        # By tak-s: connect disjointed segmentations together
                        segments = []
                        for segment in object0['segmentation']:
                            segments.extend(oneD2twoD(segment)+segments[:2])
                        segments = [segments]
                        
                        if plot: 
                            polygon_coords.append([])
                            segment_coords.append([])

                    # if crowded, use bbox
                    else: 
                        # x1, y1, x1, y2, x2, y2, x2, y1
                        label = [catid_to_idx[object0['category_id']],
                                 object0['bbox'][0], object0['bbox'][1], object0['bbox'][0], object0['bbox'][3],
                                 object0['bbox'][2], object0['bbox'][3], object0['bbox'][2], object0['bbox'][0]] 

                        label = normalize_anchors(label, img['height'], img['width'])[0] # normalize xyxyxyxy
                        anno_txt.append(label)
                        continue

                    # iterate through each segmentation
                    for segment in segments:

                        #using shapely::minimum_rotated_rectangle to convert segmentation
                        multipoint = shapely.geometry.MultiPoint(segment)
                        # polygon: class id, x1, y1, x2, y2, x3, y3, x4, y4 (unnormalized)
                        try:
                            label = [catid_to_idx[object0['category_id']],
                                     *np.array(multipoint.minimum_rotated_rectangle.exterior.coords[:-1]).ravel().tolist()]
                            label, label_pixel = normalize_anchors(label, img['height'], img['width']) # normalize xyxyxyxy
                            anno_txt.append(label)
                            if plot: 
                                polygon_coords[-1].append(np.vstack((label_pixel[1:].reshape(-1, 2), label_pixel[1:3])))
                                segment_coords[-1].append(segment)
                        except Exception as e:
                            print('Warning: Ignore label, ', e)
                if plot: 
                    polygon_plot_image(img_name, polygon_coords, segment_coords)
                    plot_now += 1
                np.savetxt(str(anno_name), np.array(anno_txt), fmt=["%i"]+["%.6f"]*8)
        with open(str(parent_path/(prefix+'.txt')), 'w+') as f:
            for img_i, img_name in enumerate(img_txt):
                if img_i == len(img_txt)-1: f.write(img_name)
                else: f.write(img_name+'\n')


def normalize_anchors(label, img_h, img_w):
    """
        polygon
        FROM class id, x1, y1, x2, y2, x3, y3, x4, y4 (unnormalized)
        TO class id (unchanged), x1, y1, x2, y2, x3, y3, x4, y4 (normalized to [0, 1])
    """
    label = np.array(label)
    label_pixel = np.copy(label)
    label[1::2] = label[1::2]/img_w
    label[2::2] = label[2::2]/img_h
    # Common out the following lines to enable: polygon corners can be out of images
    # label[1::2] = label[1::2].clip(0., img_w)/img_w
    # label[2::2] = label[2::2].clip(0., img_h)/img_h
    # label_pixel[1::2] = label_pixel[1::2].clip(0., img_w)
    # label_pixel[2::2] = label_pixel[2::2].clip(0., img_h)
    return label, label_pixel
        
def polygon_plot_image(img_name, polygon_coords, segment_coords=None):
    img = mpimg.imread(img_name)
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    if segment_coords is not None:
        for seg_coo in segment_coords:
            for segment in seg_coo:
                plt.plot(*list(zip(*segment)))
    for poly_coo in polygon_coords:
        for polygon in poly_coo:
            plt.plot(*list(zip(*polygon)))
    plt.axis('off')
    plt.show()

In [14]:

import json
import shutil

def read_json_file(path_to_file):
    with open(path_to_file, "r") as p:
        return json.load(p)

def read_labelme_anno(path_anno):
    for dir_, _, files in os.walk(path_anno):
        for file_name in files:
            if not file_name[0] == '.':
                rel_dir = os.path.relpath(dir_, path_anno)
                rel_file = os.path.join(rel_dir, file_name)
                _labelme_anno = read_json_file(rel_file)
                print(_labelme_anno)
    # for _path in os.listdir(path_anno):
    #     _anno_rel_path = os.path.relpath(_path, start="/content/gdrive/MyDrive/Work/PolygonObjectDetection-main/data_rotation/labels/train")
    #     print(_anno_rel_path)
    #     _labelme_anno = read_json_file(_anno_rel_path)
    #     print(_labelme_anno)


catidx = {"tube": 0}

def convert_annos(labels_path):
    # root dir is cwd in most cases 
    # dataset_path is the name of the dataset
    # file structure:
    #   - root
    #       - dataset_path
    #           - train
    #               - images
    #               - labels
    #               - polygon_labels
    #           - val
    #               - images
    #               - labels
    #               - polygon_labels
    # then you can manually change the name of the folders if you want
    for dataset_folder in os.listdir(labels_path):
        if not dataset_folder[0] == '.':
            # train
            polygon_anno_path = os.path.join(os.path.dirname(labels_path), "polygon_labels", dataset_folder)
            if os.path.exists(polygon_anno_path):
                shutil.rmtree(polygon_anno_path)
            os.makedirs(polygon_anno_path)
            json_dataset_folder = os.path.join(labels_path, dataset_folder)
            for anno_file in os.listdir(json_dataset_folder):
                if not anno_file[0] == '.':
                    json_path = os.path.join(json_dataset_folder, anno_file)
                    new_json_path = os.path.join(polygon_anno_path, os.path.splitext(anno_file)[0] + ".txt")
                    anno = read_json_file(json_path)
                    annos_list = []
                    for polygon in anno['shapes']:
                        coordinates = []
                        for point in polygon['points']:
                            coordinates += point 
                        assert len(coordinates) == 8, "Label does not have four points in "+json_path
                        cat = catidx[polygon['label']]
                        labels = [cat, *coordinates]
                        label, label_pixel = normalize_anchors(labels, anno["imageHeight"], anno["imageWidth"])
                        annos_list.append(label)
                    annos_array = np.array(annos_list)
                    # write to new_json_path
                    np.savetxt(str(new_json_path), annos_array, fmt=["%i"]+["%.6f"]*8)



anno = convert_annos("/Users/antoniomorais/Work/PolygonObjectDetection/data_rotation/labels")





In [72]:


label, label_pixel = normalize_anchors(labels, 1942, 2590)

NameError: ignored